In [1]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('Dataset/general_data.csv')

In [2]:
from sklearn import preprocessing as pp

df = dataset
df['Attrition']      = pp.LabelEncoder().fit_transform(df['Attrition'])
df['BusinessTravel'] = pp.LabelEncoder().fit_transform(df['BusinessTravel'])
df['Department']     = pp.LabelEncoder().fit_transform(df['Department'])
df['EducationField'] = pp.LabelEncoder().fit_transform(df['EducationField'])
df['Gender']         = pp.LabelEncoder().fit_transform(df['Gender'])
df['JobRole']        = pp.LabelEncoder().fit_transform(df['JobRole'])
df['MaritalStatus']  = pp.LabelEncoder().fit_transform(df['MaritalStatus'])


In [3]:
df1 = df.drop(['EmployeeCount','EmployeeID','Over18','StandardHours'], axis=1)

df1 = df1.dropna()
df2 = df1.drop_duplicates()

df2['TotalWorkingYears'] = np.round(df1['TotalWorkingYears'])
df2['MonthlyIncome']     = np.round(df1['MonthlyIncome'])

df2.columns

C:\Users\Joy\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Joy\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole',
       'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [4]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

features = ['Age','BusinessTravel','Department','DistanceFromHome','Education','EducationField','Gender','JobLevel','JobRole','MaritalStatus','MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','StockOptionLevel','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsSinceLastPromotion','YearsWithCurrManager']

rf_model.fit(X=df2[features],y=df2["Attrition"])

RandomForestClassifier(max_features=2, n_estimators=1000, oob_score=True)

In [5]:
print("OOB Accuracy score: ", rf_model.oob_score_)

OOB Accuracy score:  0.8435374149659864


In [20]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature, "    \t:", imp)

Age      	: 0.09852488510388213
BusinessTravel      	: 0.02770405623412496
Department      	: 0.026328463168833662
DistanceFromHome      	: 0.06818625439712718
Education      	: 0.039904481182891194
EducationField      	: 0.041648401211808225
Gender      	: 0.0186656353184396
JobLevel      	: 0.03755174955068577
JobRole      	: 0.054915046948035146
MaritalStatus      	: 0.03912653520472101
MonthlyIncome      	: 0.09366782127599364
NumCompaniesWorked      	: 0.05586304925372595
PercentSalaryHike      	: 0.06561988914637865
StockOptionLevel      	: 0.03359604522747235
TotalWorkingYears      	: 0.08696108257433229
TrainingTimesLastYear      	: 0.04418301437360989
YearsAtCompany      	: 0.06843071882564983
YearsSinceLastPromotion      	: 0.04434878603852514
YearsWithCurrManager      	: 0.054774084963763514


#### The variables with higher importance value :
##### ______1. Age
##### ______2. MonthlyIncome
##### ______3. TotalWorkingYears  

#### Taking these variables to fit in the model 

In [21]:
from sklearn import tree

tree_model = tree.DecisionTreeClassifier(max_depth=6, max_leaf_nodes=12)

cl_data = pd.DataFrame([df2["Age"],df2["MonthlyIncome"],df2["TotalWorkingYears"]]).T

tree_model.fit(X=cl_data,y=df2["Attrition"])

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=12)

In [22]:
with open("Dtree_Attrition.dot","w") as f:
    f=tree.export_graphviz(tree_model,feature_names=["Age","MonthlyIncome","TotalWorkingYears"],out_file=f)

### Causes for attrition occurances

##### 1. Person having experience less than 1.5 years and age is less than 33.5 then attrition occurance is high.
##### 2. Person having experience less than 1.5 years and age is less than 33.5 and Monthly Income < 112610 then attrition occurance is high.
##### 3. Person having experience less than 1.5 years and age is less than 23.5 and Monthly Income < 23140 then attrition occurance is high.
##### 4. Person having experience less than 1.5 years and age is less than 18.5 and Monthly Income < 32530 then attrition occurance is high.
##### 5. Person having experience more than 1.5 years and age is more than 33.5 then attrition occurance is low.
##### 6. Person having experience more than 39 years and  and Monthly Income < 10300 then attrition occurance is high.
##### 7. Person having experience more than 5.5 years and  and Monthly Income > 10300 then attrition occurance is low.